In [2]:
# Gerekli Kütüphaneler
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
import gc # RAM temizliği için (Garbage Collector)

## 1. Veriyi Yükle

In [3]:
def load_data(path):
    df = pd.read_csv(path)
    # ID sütunları gereksiz o yüzden kaldıralım.
    df = df.drop(columns=['Id', 'groupId', 'matchId'])
    return df

print("Loading data...")
df = load_data("../pubg_predictor/data/train_V2.csv")
print("Data loaded successfully.")

Loading data...
Data loaded successfully.


## 2. Veriyi İşlemek
"matchType" değişkenini (squad, duo, solo) sayıya çevirmeliyiz ki makine bunu anlayabilsin. 

In [4]:
print("Categorical encoding for matchType...")
df = pd.get_dummies(df, columns=['matchType'])

x = df.drop(columns=['winPlacePerc'])
y = df['winPlacePerc']

# RAM temizliği
del df
gc.collect()

Categorical encoding for matchType...


40

## 3. Baseline Model Eğitimi
Verimizi %80 Eğitim, %20 Test olarak eğiteceğiz.

In [9]:
# Train-Test Split
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

# Modeli kuralım (Hızlı olduğu için LightGBM kullanıyoruz)
model = LGBMRegressor(n_estimators=100, learning_rate=0.05, n_jobs=-1)

print("Model training...")
model.fit(X_train, y_train)

# Tahmin
print("Making predictions...")
preds = model.predict(X_val)

# Başarı Skoru (Mean Absolute Error - Hata Payı)
mae = mean_absolute_error(y_val, preds)
print("-" * 30)
print(f"📉 BASELINE MAE SCORE: {mae:.5f}")
print("-" * 30)

Model training...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.082440 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2833
[LightGBM] [Info] Number of data points in the train set: 3557572, number of used features: 40
[LightGBM] [Info] Start training from score 0.472873
Making predictions...
------------------------------
📉 BASELINE MAE SCORE: 0.06367
------------------------------


## 4. Skor Yorumu
Modelimiz, bir oyuncunun sıralamasını (0 ile 1 arasında) tahmin ederken ortalama %6.3 gibi bir düşük hata payı ile çalışıyor. Kısaca modelimiz rastgele tahminler yürütmüyor, mantıklı tahminler yürütüyor.